In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
import json
import re
import string
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#Import the necessary methods from tweepy library  

#install tweepy if you don't have it
#!pip install tweepy
import json
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#sentiment analysis package
#!pip install textblob
from textblob import TextBlob

#general text pre-processor
#!pip install nltk
import nltk
from nltk.corpus import stopwords

#tweet pre-processor 
#!pip install tweet-preprocessor
import preprocessor as p

In [4]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [6]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag = 'h2')
res

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)',
 '96. James Copnall (@JamesCopnall)',
 '95. Online Africa (@oafrica)',
 '94. Patrick Ngowi (@PatrickNgowi)',
 '93. DOS African Affairs (@StateAfrica)',
 '92. MoadowAJE (@Moadow)',
 '91. Brendan Boyle (@BrendanSAfrica)',
 '90. City of Tshwane (@CityTshwane)',
 '89. VISI Magazine (@VISI_Mag)',
 '88. andBeyond (@andBeyondSafari)',
 '87. This Is Africa (@ThisIsAfricaTIA)',
 '86. Sarah Carter (@sarzss)',
 '85. The EIU Africa team (@TheEIU_Africa)',
 '84. Investing In Africa (@InvestInAfrica)',
 '83. Barry Malone (@malonebarry)',
 '82. ARTsouthAFRICA (@artsouthafrica)',
 '81. Kahn Morbee (@KahnMorbee)',
 '80. Jamal Osman (@JamalMOsman)',
 '79. iamsuede™ (@iamsuede)',
 '78. Mike Stopforth (@mikestopforth)',
 '77. Equal Education (@equal_education)',
 '76. Tristan McConnell (@t_mcconnell)',
 '75. Kate Forbes (@forbeesta)',
 '74. Vanessa Raphaely (@hur

In [8]:
res2 = pd.DataFrame(res, columns= ['Influencer']) #Turn the list into a dataframe

In [9]:
df = res2[res2.Influencer.str.contains('@')] #filter out items in the list not needed

In [10]:
#Use regex to grab the handle
df['handle'] = df.Influencer.apply(lambda x: re.compile('\((.*?)\)').search(x).group(1)) 
df.Influencer = df.Influencer.apply(lambda x: re.sub('\((.*?)\)','',x))

#split and grab the rank (based on the website's ranking)
df['rank'] = df.Influencer.apply(lambda x: int(x.split('.')[0]))
df.Influencer = df.Influencer.apply(lambda x: re.sub(r'[0-9]+\.', '',x))

In [11]:
df = df.sort_values(by = 'rank').reset_index(drop = True) ; df.head() #reset index and print first 5 rows to screen

,Influencer,handle,rank
0,Trevor Noah,@Trevornoah,1
1,Gareth Cliff,@GarethCliff,2
2,Jacob G. Zuma,@SAPresident,3
3,News24,@News24,4
4,Julius Sello Malema,@Julius_S_Malema,5


In [12]:
#df.to_csv('AfricanInfluencers.csv', index = False) #save to disk

In [13]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [14]:
resUrl = get_elements(response, search={'find_all':{'class_':'wp-block-embed__wrapper'}})
resUrl

findaing all of {'class_': 'wp-block-embed__wrapper'}


['\nhttps://twitter.com/TsholetsaDomi/status/1238324860536922112\n',
 '\nhttps://twitter.com/Azali_officiel/status/1239649350747332613\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees' unions to map a collaborative effort in the fight against <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a>. <a href="https://t.co/EIYNGOEKRN">pic.twitter.com/EIYNGOEKRN</a></p>— Eswatini Government (@EswatiniGovern1) <a href="https://twitter.com/EswatiniGovern1/status/1241038139889721346?ref_src=twsrc%5Etfw">March 20, 2020</a></blockquote>,
 <script async="" charset="utf-8" src="https://platform.twitter.com/widgets.js"></script>,
 '\n',
 '\nhttps://twitter.com/SE_Rajoelina/status/1241101811647500288\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">GUIDEL

In [18]:
#split by blockquote class which holds the header that we need
resUrl1 = pd.DataFrame(str(resUrl).split('<blockquote class='), columns=['wrapper'])

In [19]:
resUrl1 = resUrl1[resUrl1.wrapper.str.contains('@')] #filter out all rows not needed
resUrl1.reset_index(drop = True, inplace= True)

In [20]:
#Use regex to grab information enclosed in the </p> tag
resUrl1['GetTag'] = resUrl1.wrapper.apply(lambda x: re.compile('\</p\>(.*?)\<a').search(x).group(1))

#grab the Government handle (searching for any word that begins with @)
resUrl1['handle'] = resUrl1.wrapper.apply(lambda x: re.compile(r'[@]\S*').search(x).group(0).strip(')'))

#Get the screenname
resUrl1['ScreenName'] = resUrl1.GetTag.apply(lambda x: re.sub('\((.*?)\)','',x).strip('— '))

In [21]:
resUrl1

,wrapper,GetTag,handle,ScreenName
0,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Eswatini Government (@EswatiniGovern1),@EswatiniGovern1,Eswatini Government
1,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Malawi Government (@MalawiGovt),@MalawiGovt,Malawi Government
2,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Hage G. Geingob (@hagegeingob),@hagegeingob,Hage G. Geingob
3,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Seychelles Ministry of Finance (@FinanceSC),@FinanceSC,Seychelles Ministry of Finance
4,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— PresidencyZA (@PresidencyZA),@PresidencyZA,PresidencyZA
5,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Ministry of Health Zambia (@mohzambia),@Dora_Siliya</a>,Ministry of Health Zambia
6,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— President of Zimbabwe (@edmnangagwa),@edmnangagwa,President of Zimbabwe
7,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— MinSantédj (@MinSantedj),@MinSantedj,MinSantédj
8,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— Yemane G. Meskel (@hawelti),@hawelti,Yemane G. Meskel
9,"""twitter-tweet"" data-dnt=""true"" data-width=""55...",— State House Kenya (@StateHouseKenya),@StateHouseKenya,State House Kenya


In [22]:
Govthandle = resUrl1[['ScreenName', 'handle']]

In [23]:
Govthandle.head()

,ScreenName,handle
0,Eswatini Government,@EswatiniGovern1
1,Malawi Government,@MalawiGovt
2,Hage G. Geingob,@hagegeingob
3,Seychelles Ministry of Finance,@FinanceSC
4,PresidencyZA,@PresidencyZA


In [ ]:
# Govthandle.to_csv('TopGovHandle.csv', index = False)